## Imports

In [2]:
import pandas as pd
import numpy as np
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import re
import glob
import os
from datetime import date
import requests
import time

### For Google Sheets ###
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json

## Scraper

In [3]:
### Find local path
path = os.getcwd()

In [4]:
### Get a list of all CSV files in current directory
filelist = glob.glob(os.path.join(path,'*.csv'))

In [5]:
### Read in old dataframe
for f in filelist:
    if re.search('ALL_DEEDS',f):
        old_df = pd.read_csv(f)

In [6]:
### Remove any CSV files in current directory
for f in filelist:
    os.remove(f)

Web link: https://crs.cookcountyclerkil.gov/Search/Additional

## Main Doc Scraper
(Deeds & mortgages, those with a consideration amount)

In [7]:
### Set up start date, end date, and doc list

start_date = '01012022'
end_date = '03132023'

run_date = date.today().strftime('%b-%d-%Y')

doc_list = [
    'DEED',
    'DEIT',
    'QCD',
    'SPWD',
    'TRUD',
    'TEED',
    'WARD',
    'MORT'
]

Target website: https://crs.cookcountyclerkil.gov/Search/Additional

In [8]:
def page_scraper(page, page_counter):

    dfs = pd.read_html(page)
    df = dfs[0]

    soup = BeautifulSoup(page, 'html.parser')

    link_suffix = 'https://crs.cookcountyclerkil.gov/'

    deed_urls = []
    for link in soup.find_all('a',attrs={'href': re.compile('^/Document/Detail')}):
        page = link_suffix + link.get('href')
        deed_urls.append(page)

    df['deed_urls'] = deed_urls

    df['Consi. Amt.'] = df['Consi. Amt.'].str.replace('$','',regex=False)
    df['Consi. Amt.'] = df['Consi. Amt.'].str.replace(',','',regex=False)

    df['Consi. Amt.'] = pd.to_numeric(df['Consi. Amt.'])

    page_counter = page_counter + 1

    df.to_csv(f'{doc}_page{page_counter}_{start_date}_to_{end_date}.csv')
    
    return page_counter

In [9]:
### Run main doc scraper

for doc in doc_list:
    
    page_counter = 0

    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = True)
    page = await browser.new_page()

    # Go to https://crs.cookcountyclerkil.gov/Search/Additional
    await page.goto("https://crs.cookcountyclerkil.gov/Search/Additional")

    # Click text=Document Type Search
    await page.locator("text=Document Type Search").click()

    # Select DEED
    await page.locator("text=Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER ON DEATH INSTRUMEN A >> select[name=\"DocumentType\"]").select_option(doc)

    # Click text=From Date * (mm/dd/yyyy) >> input[name="RecordedFromDate"]
    await page.locator("text=From Date * (mm/dd/yyyy) >> input[name=\"RecordedFromDate\"]").click()

    await page.locator("text=From Date * (mm/dd/yyyy) >> input[name=\"RecordedFromDate\"]").fill(start_date)

    # Click text=To Date * (mm/dd/yyyy) >> input[name="RecordedToDate"]
    await page.locator("text=To Date * (mm/dd/yyyy) >> input[name=\"RecordedToDate\"]").click()

    await page.locator("text=To Date * (mm/dd/yyyy) >> input[name=\"RecordedToDate\"]").fill(end_date)

    # Click input[name="LowerLimit"]
    await page.locator("input[name=\"LowerLimit\"]").click()

    # Fill input[name="LowerLimit"]
    await page.locator("input[name=\"LowerLimit\"]").fill("4000000")

    # Click text=Document Type Search Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER >> button[name="submitButton"]
    await page.locator("text=Document Type Search Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER >> button[name=\"submitButton\"]").click()

    try:
        
        await page.wait_for_selector("div[class=table-responsive]")

        x = await page.content()
        
        try:
        
            page_scraper(x, page_counter)
            
        except Exception as e:
            2 + 2
            
        i = 1
        
        while i < 100:
            
            try:
                
                await page.wait_for_selector("div[class=table-responsive]")

                await page.locator("text=»").click()
                
                y = await page.content()

                page_scraper(y, i)
                
                i += 1
                
            except Exception as e:
                
                i += 1000
                
                await page.wait_for_selector("div[class=table-responsive]")
                
                y = await page.content()

                page_scraper(y, i)
        
        await browser.close()

    # ---------------------
#         await browser.close()
        
    except Exception as e:
        2 + 2

## Alternate Doc Scraper
(Bankruptcy and Lis Pendens)

In [10]:
alt_doc_list = [
    'FORF',
    'LISP',
    'LISF'
]

In [11]:
def alt_page_scraper(page, page_counter):

    dfs = pd.read_html(page)
    df = dfs[0]

    soup = BeautifulSoup(page, 'html.parser')

    link_suffix = 'https://crs.cookcountyclerkil.gov/'

    deed_urls = []
    for link in soup.find_all('a',attrs={'href': re.compile('^/Document/Detail')}):
        page = link_suffix + link.get('href')
        deed_urls.append(page)

    df['deed_urls'] = deed_urls

    page_counter = page_counter + 1

    df.to_csv(f'{doc}_page{page_counter}_{start_date}_to_{end_date}.csv')
    
    return page_counter

In [12]:
### Run alt doc scraper

for doc in alt_doc_list:
    
    page_counter = 0

    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = True)
    page = await browser.new_page()

    # Go to https://crs.cookcountyclerkil.gov/Search/Additional
    await page.goto("https://crs.cookcountyclerkil.gov/Search/Additional")

    # Click text=Document Type Search
    await page.locator("text=Document Type Search").click()

    # Select DEED
    await page.locator("text=Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER ON DEATH INSTRUMEN A >> select[name=\"DocumentType\"]").select_option(doc)

    # Click text=From Date * (mm/dd/yyyy) >> input[name="RecordedFromDate"]
    await page.locator("text=From Date * (mm/dd/yyyy) >> input[name=\"RecordedFromDate\"]").click()

    await page.locator("text=From Date * (mm/dd/yyyy) >> input[name=\"RecordedFromDate\"]").fill(start_date)

    # Click text=To Date * (mm/dd/yyyy) >> input[name="RecordedToDate"]
    await page.locator("text=To Date * (mm/dd/yyyy) >> input[name=\"RecordedToDate\"]").click()

    await page.locator("text=To Date * (mm/dd/yyyy) >> input[name=\"RecordedToDate\"]").fill(end_date)

    # Click text=Document Type Search Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER >> button[name="submitButton"]
    await page.locator("text=Document Type Search Document Type * ABROGATION ACCEPTANCE ACCEPTANCE OF TRANFER >> button[name=\"submitButton\"]").click()

    try:
        
        await page.wait_for_selector("div[class=table-responsive]")

        x = await page.content()
        
        try:
        
            page_scraper(x, page_counter)
            
        except Exception as e:
            print(f'Function failed: {e}')
            
        i = 1
        
        while i < 100:
            
            try:
                
                await page.wait_for_selector("div[class=table-responsive]")

                await page.locator("text=»").click()
                
                y = await page.content()

                page_scraper(y, i)
                
                i += 1
                
            except Exception as e:
                
                i += 1000
                
                await page.wait_for_selector("div[class=table-responsive]")
                
                y = await page.content()

                page_scraper(y, i)
        
        await browser.close()

    # ---------------------
#         await browser.close()
        
    except Exception as e:
        2 + 2

Function failed: Can only use .str accessor with string values!
Function failed: Can only use .str accessor with string values!
Function failed: Length of values (183) does not match length of index (100)


## Create Master CSV file

In [13]:
### Join all created CSVs into one file

all_csvs = glob.glob(os.path.join(path,'*.csv'))

li = []

for filename in all_csvs:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)
    
df = pd.concat(li, axis=0, ignore_index=True)

In [14]:
### Clean master CSV and edit data types

df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0','View Doc'])
df['Consi. Amt.'] = df['Consi. Amt.'].apply(lambda x : '${:,}'.format(x))

df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'])
df = df.sort_values(by='Doc Recorded', ascending=False)

df = df.drop_duplicates()

In [15]:
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 1600)

## Get Links to docs

In [16]:
deed_list = df['deed_urls'].to_list()

In [17]:
###
# x[1] = Grantor table
# x[2] = Grantee table
# x[3] = Legal Description and Subdivision table
###

grantor_list = []
grantee_list = []
PIN_list = []
url_counter = 0

for deed in deed_list:
    page = requests.get(deed)
    x = pd.read_html(page.content)
    
    try:
        x_1 = x[1]
        x_1 = x_1.drop('Trust#',axis=1)
        x_1 = pd.DataFrame({'Name': [', '.join(x_1['Name'].str.strip('"').tolist())]})
        x_1 = x_1['Name'].to_list()
        grantor_list.append(x_1)
    except Exception as e:
        2 + 2
    
    try:
        x_2 = x[2]
        x_2 = x_2.drop('Trust#',axis=1)
        x_2 = pd.DataFrame({'Name': [', '.join(x_2['Name'].str.strip('"').tolist())]})
        x_2 = x_2['Name'].to_list()
        grantee_list.append(x_2)
    except Exception as e:
        2 + 2
    
    try:
        x_3 = x[3]
        x_3 = x_3.filter(items=['Property Index # (PIN)'])
        x_3 = pd.DataFrame({'PIN': [', '.join(x_3['Property Index # (PIN)'].str.strip('"').tolist())]})
        x_3 = x_3['PIN'].to_list()
        PIN_list.append(x_3)
    except Exception as e:
        PIN_list.append('No PIN found')
        
    url_counter += 1
    time.sleep(1)

In [18]:
df['grantor_all'] = grantor_list
df['grantee_all'] = grantee_list
df['PIN_all'] = PIN_list

In [19]:
df = df.reset_index()

In [20]:
x = f'ALL_DEEDS_{start_date}_to_{end_date}_run_{run_date}.csv'
df.to_csv(x)

In [21]:
df_deed = df.loc[df['Doc Type'] != 'MORTGAGE'].reset_index()
df_deed = df_deed.drop(columns=['level_0','index'])

## Google Sheets

In [22]:
scopes = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]

credentials = ServiceAccountCredentials.from_json_keyfile_name("autoscraper-380600-d911f955a585.json", scopes) #access the json key you downloaded earlier 
file = gspread.authorize(credentials) # authenticate the JSON key with gspread
sheet = file.open("CookCountyScraper") #open sheet
sheet = sheet.sheet1 

In [23]:
df = pd.read_csv(x)

In [24]:
df = df.drop(columns=['Unnamed: 0','index'])

In [27]:
df = df.replace([pd.np.inf, -pd.np.inf, pd.np.nan], 'NA')

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_53968/593088941.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df = df.replace([pd.np.inf, -pd.np.inf, pd.np.nan], 'NA')


In [25]:
# Clear existing data (optional)
sheet.clear()

{'spreadsheetId': '1J-kWQ3SabAKz44NNYA3H7OE4NfMxdmk4AND2z_mh-AQ',
 'clearedRange': 'Sheet1!A1:Z1249'}

In [26]:
header = df.columns.tolist()
data = df.values.tolist()
sheet.insert_row(header, 1)
sheet.insert_rows(data, 2)

InvalidJSONError: Out of range float values are not JSON compliant